In [1]:
#!g1.1

import numpy as np 
import random
from collections import namedtuple
import time

import torch
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torch.optim import SGD, Adam

import tqdm


from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck

from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA

%matplotlib inline

In [2]:
#!g1.1

DATA_DIR = 'imagenet-r'

NUM_CLASSES = 200
DEVICE = 'cuda'
N = 30000

SEED = 42 

np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
None

In [3]:
#!g1.1
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomCrop(196),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


dataset = datasets.ImageFolder(DATA_DIR, transform=val_transforms)
trainset, testset = random_split(dataset, lengths=[27000, 3000])

In [4]:
#!g1.1

train_loader = DataLoader(trainset, pin_memory=False, shuffle=True,  batch_size=128)
test_loader  = DataLoader(testset,  pin_memory=False, shuffle=False, batch_size=100)

# Dropout

In [5]:
#!g1.1
def train(model, train_loader, test_loader, num_epochs=30, learning_rate=0.1):
    optimizer = Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(1, num_epochs + 1):
        print(f'Epoch {epoch}:')
        start_time = time.time()
        
        model.train()
        train_loss = 0
        train_acc = 0
        for i, (X, y) in enumerate(train_loader):
            optimizer.zero_grad()
            
            X = X.to(DEVICE)
            y = y.to(DEVICE)
            
            y_pred = model.forward(X)
            loss   = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            
            acc = (y == y_pred.argmax(dim=-1)).float().mean().item()
            
            train_acc  += (acc               - train_acc)  / (i + 1)
            train_loss += (loss.cpu().item() - train_loss) / (i + 1)
            if (i + 1) % 10 == 0:
                print(f'\rtrain_acc {train_acc:.4f} train_loss {train_loss:.4f}', end='')
        print()
        model.eval()
        test_loss = 0
        test_acc  = 0 
        for i, (X, y) in enumerate(test_loader):
            X = X.to(DEVICE)
            y = y.to(DEVICE)
            
            y_pred = model.forward(X)
            loss   = criterion(y_pred, y)
            
            acc = (y == y_pred.argmax(dim=-1)).float().mean().item()
            
            test_acc  += (acc               - test_acc)  / (i + 1)
            test_loss += (loss.cpu().item() - test_loss) / (i + 1)
            
        print(f'test_acc {test_acc:.4f} test_loss {test_loss:.4f}')
        print(f'took {time.time() - start_time:.0f}s')
        print()

In [ ]:
#!g1.1

resnet18 = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=NUM_CLASSES)
resnet18.fc = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(512 * BasicBlock.expansion, NUM_CLASSES)
)
model = resnet18.to(DEVICE)
train(model, train_loader, test_loader, num_epochs=100, learning_rate=0.03)

Epoch 1:
train_acc 0.0170 train_loss 5.3877
test_acc 0.0193 test_loss 5.1079
took 406s

Epoch 2:
train_acc 0.0207 train_loss 5.1080
test_acc 0.0277 test_loss 5.0699
took 320s

Epoch 3:
train_acc 0.0257 train_loss 5.0673
test_acc 0.0283 test_loss 5.0026
took 322s

Epoch 4:
train_acc 0.0315 train_loss 5.0217
test_acc 0.0347 test_loss 4.9499
took 323s

Epoch 5:
train_acc 0.0347 train_loss 4.9713
test_acc 0.0527 test_loss 4.8926
took 321s

Epoch 6:
train_acc 0.0427 train_loss 4.9239
test_acc 0.0517 test_loss 4.8927
took 322s

Epoch 7:
train_acc 0.0486 train_loss 4.8656
test_acc 0.0553 test_loss 4.8220
took 321s

Epoch 8:
train_acc 0.0538 train_loss 4.8023
test_acc 0.0630 test_loss 4.7882
took 320s

Epoch 9:
train_acc 0.0600 train_loss 4.7367
test_acc 0.0667 test_loss 4.7292
took 321s

Epoch 10:
train_acc 0.0672 train_loss 4.6783
test_acc 0.0693 test_loss 4.7277
took 320s

Epoch 11:
train_acc 0.0719 train_loss 4.6161
test_acc 0.0783 test_loss 4.6803
took 320s

Epoch 12:
train_acc 0.0816 tra